# Model demo

This notebook is meant to be a quick entry into ViperGPT. **Warning:** This notebook will execute arbitrary code on your machine. Proceed at your own risk.

Before running this code, modify any parameters at `configs/my_config.yaml`. For example, you may want to change the BLIP model (`blip_v2_model_type`) from XXL to XL if your GPU does not have enough memory.

In [ ]:
%load_ext autoreload
%autoreload 2
import torch
print(torch.cuda.is_available())
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

import os
os.environ["PATH"] += os.pathsep + "/usr/local/cuda/bin"
os.environ["LD_LIBRARY_PATH"] = "/usr/local/cuda/lib64"

In [ ]:
from main_simple_lib import *

In [ ]:
import openai 
openai.Model.list()

In [ ]:
from vision_processes import consumers
model_name="codex"
input_type="video"

In [ ]:
consumers.keys()

In [ ]:
%%timeit -r 1 -n 1 
# Let's try a couple of consumers
query = 'In how many frames are both paws on the wheel?'
out = consumers[model_name](prompt=query, input_type=input_type)

In [ ]:
%%timeit -r 1 -n 1 
# when the prompt does not have videosegment class 6 sec
# when the prompt does have videosegment class? over 20 sec over 4097 tokens

get_code(query)

In [ ]:
# Even though the length of the sentence is short, we cannot process this sentence.
# "In how many frames are both paws in the wheel?" This model's maximum context length is 4097 tokens. However, you requested 4548 tokens (4036 in the messages, 512 in the completion). Please reduce the length of the messages or completion.
%% timeit -r 1 -n 1
query ="How many frames have both paws in the wheel?"
code = forward("codex", prompt=query, input_type="video")

In [ ]:
%timeit 
query ="How many frames have both paws in the wheel?"
code = forward("codex", prompt=query, input_type="image")

In [ ]:
%timeit 
query ="How many frames have both paws in the wheel?"
code = forward("codex", prompt=query, input_type="video")

In [ ]:
if config.codex.model not in ('gpt-3.5-turbo', 'gpt-4'):
    code = f'def execute_command(image, my_fig, time_wait_between_lines, syntax):' + code # chat models give execute_command due to system behaviour
code_for_syntax = code.replace("(image, my_fig, time_wait_between_lines, syntax)", "(image)")
syntax_1 = Syntax(code_for_syntax, "python", theme="monokai", line_numbers=True, start_line=0)
console.print(syntax_1)


In [ ]:
code = ast.unparse(ast.parse(code))
code_for_syntax_2 = code.replace("(image, my_fig, time_wait_between_lines, syntax)", "(image)")
syntax_2 = Syntax(code_for_syntax_2, "python", theme="monokai", line_numbers=True, start_line=0)


In [ ]:
# occlussion?
# fast movements?
# more realistic videos


In [ ]:
video_path = "/home/ekellbuch/vits/viper/data/ibl1/ibl1/labeled-data/ibl1"

im = load_image(im_path)
query = 'Are there two paws in each frame?'

show_single_image(im)
code = get_code(query)

In [ ]:
output = execute_code(code, im, show_intermediate_steps=True)

In [ ]:
class ImageFolderDataset(Dataset):
    def __init__(self, folder_path, transform=None):
        self.folder_path = folder_path
        self.image_list = os.listdir(folder_path)
        self.transform = transform

    def __len__(self):
        return len(self.image_list)

    def __getitem__(self, index):
        image_path = os.path.join(self.folder_path, self.image_list[index])
        image = Image.open(image_path).convert("RGB")

        if self.transform is not None:
            image = self.transform(image)

        return image
    
def load_video(folder_path):
    transform = transforms.Compose([
    transforms.ToTensor(),
])
    dataset = ImageFolderDataset(folder_path, transform=transform)
    #batch_size = 32 
    #data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    tensor_sizes = []
    for image_tensor in dataset:
        tensor_size = image_tensor#.size()
        tensor_sizes.append(tensor_size)
    return tensor_sizes
    
    